In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

import statsmodels.api as sm
import statsmodels.tsa.arima_process as apro
from sklearn.metrics import r2_score

# Mean reversion with daily prices

In [6]:
with open("sp500_list.txt", "r") as f:
    sp500_list = [line.strip() for line in f]

In [11]:
rfdata = pd.read_csv("market_data.csv", index_col=0, low_memory=False)
rfdata = rfdata.drop(rfdata.index[[0, 1]])
rfdata.columns = pd.MultiIndex.from_arrays(
    [["Adj Close"] * len(sp500_list) + ["Volume"] * len(sp500_list),
     sp500_list * 2]
)
rfdata.index = pd.to_datetime(rfdata.index)
rfdata = rfdata.astype(float)

data = rfdata["Adj Close"]
data = data.dropna(axis=1)

data.head()

,A,AAL,AAPL,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,...,WST,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA
2012-01-03,23.452354,4.826836,12.433825,21.276012,12.493333,42.139866,28.570000,27.088427,20.861462,36.140427,...,17.623835,81.378044,12.017909,84.649750,18.264717,52.614315,21.305161,33.329346,47.259995,35.720001
2012-01-04,23.265926,4.741990,12.500645,21.193491,12.266667,42.123951,28.280001,27.050842,21.208067,36.100613,...,17.457529,80.916603,11.885774,83.194000,18.144459,52.626541,22.008551,33.556980,46.620407,35.450001
2012-01-05,23.786655,5.156796,12.639430,21.144733,12.500000,41.320160,28.480000,27.171131,21.114191,36.366077,...,17.476002,82.321846,11.816563,82.324966,18.224625,52.467487,21.694986,33.813053,47.172401,35.400002
2012-01-06,24.043812,5.279351,12.771558,20.953428,12.486667,41.248528,28.719999,26.990690,20.991438,36.419163,...,17.531435,81.629684,11.967570,79.428307,18.171183,52.075928,21.330578,34.057732,47.295048,35.110001
2012-01-09,24.673836,5.392481,12.751301,20.949678,12.396667,41.176899,28.530001,27.516968,20.875900,36.279797,...,17.568396,80.937576,11.822856,79.205482,18.184542,52.308403,21.415331,33.926857,47.645531,34.950001


In [13]:
%%time

train_slice = slice("2013-01-01", "2023-01-01")
test_slice  = slice("2023-01-01", None)

rets = data.pct_change()
lag1 = rets.shift(1)

score_train_lin, score_test_lin, bench, p_values, corr = [], [], [], [], []
lin_pred = pd.DataFrame(index=rets.loc[test_slice].index, columns=data.columns, dtype=float)

for col in data.columns:
    df_train = pd.concat(
        {"y": rets.loc[train_slice, col], "x": lag1.loc[train_slice, col]}, axis=1
    ).dropna()
    X_train = sm.add_constant(df_train["x"], has_constant="add")
    y_train = df_train["y"]
    model = sm.OLS(y_train.to_numpy(), X_train.to_numpy()).fit()

    df_test = pd.concat(
        {"y": rets.loc[test_slice, col], "x": lag1.loc[test_slice, col]}, axis=1
    ).dropna()
    X_test = sm.add_constant(df_test["x"], has_constant="add")
    y_test = df_test["y"]
    preds = model.predict(X_test)

    score_train_lin.append(model.rsquared)
    score_test_lin.append(r2_score(y_test, preds))
    bench.append(r2_score(y_test, np.zeros(len(y_test))))
    p_values.append(model.pvalues[1])
    corr.append(y_test.corr(pd.Series(preds, index=y_test.index)))
    lin_pred.loc[y_test.index, col] = preds

lin_score = pd.DataFrame(
    {
        "score_train": score_train_lin,
        "score_test": score_test_lin,
        "benchmark": bench,
        "p_value": p_values,
        "correlation": corr,
    },
    index=data.columns,
)


In [14]:
lin_score.sort_values(by='score_train', ascending=False).mean()

score_train    0.003966
score_test    -0.005593
benchmark     -0.004387
p_value        0.186823
correlation    0.007811
dtype: float64

In [15]:
lin_score.to_csv("LIN1score.csv")
lin_pred.to_csv("LIN1pred.csv")

An example

In [16]:
TXN = data["TXN"].pct_change().to_frame()
TXN.columns = ["Return"]
TXN["Ret-1"] = TXN["Return"].shift(1)
TXN

,Return,Ret-1
2012-01-03,NaN,NaN
2012-01-04,-0.006384,NaN
2012-01-05,0.007102,-0.006384
2012-01-06,0.000000,0.007102
2012-01-09,0.012760,0.000000
...,...,...
2023-12-22,0.008573,0.009868
2023-12-26,0.015276,0.008573
2023-12-27,0.002459,0.015276
2023-12-28,0.002862,0.002459


In [17]:
y = TXN["Return"]["2013-01-01":"2023-01-01"]
X = TXN["Ret-1"]["2013-01-01":"2023-01-01"]

In [18]:
X = sm.add_constant(X)  # Adding a constant term to the predictor
regression_model = sm.OLS(np.array(y), np.array(X))
regression_result = regression_model.fit()

print(regression_result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     75.95
Date:                Mon, 22 Sep 2025   Prob (F-statistic):           5.17e-18
Time:                        10:35:49   Log-Likelihood:                 6698.0
No. Observations:                2518   AIC:                        -1.339e+04
Df Residuals:                    2516   BIC:                        -1.338e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0011      0.000      3.189      0.0

In [19]:
regression_result.pvalues[1]

5.173171330007896e-18

In [20]:
test_targets = TXN["Ret-1"]["2023-01-01":]
test_features = sm.add_constant(TXN["Return"]["2023-01-01":])

In [21]:
r2_score(test_targets,regression_result.predict(test_features))

-0.01341155506655256

In [22]:
test_targets.corr(regression_result.predict(test_features))

0.05139722489908582

# With weekly prices

In [17]:
%%time

score_train_lin = []
score_test_lin = []
bench= []
p_values = []
corr = []
lin_pred = pd.DataFrame(columns=data.columns)

for i in data.columns:
    frame = data[i].resample('W').last().pct_change().to_frame()
    frame.columns = ["Return"]
    frame["Ret-1"] = frame["Return"].shift(1)
    
    y = frame["Return"]["2013-01-11":"2023-01-02"]
    X = frame["Ret-1"]["2013-01-11":"2023-01-02"]
    X = sm.add_constant(X)  # Adding a constant term to the predictor
    regression_model = sm.OLS(np.array(y), np.array(X))
    regression_result = regression_model.fit()
    
    test_targets = frame["Return"]["2023-01-02":]
    test_features = sm.add_constant(frame["Ret-1"]["2023-01-02":])
    
    score_test_lin.append(r2_score(test_targets,regression_result.predict(test_features)))
    score_train_lin.append(regression_result.rsquared)
    bench.append(r2_score(test_targets, [0] * len(test_targets)))
    p_values.append(regression_result.pvalues[1])                                              
                                                  
    lin_pred[i] = list(regression_result.predict(test_features))
    corr.append(test_targets.corr(regression_result.predict(test_features)))
    

lin_score = pd.DataFrame({"score_train" : score_train_lin, "score_test": score_test_lin, "benchmark" : bench, "p_value": p_values, "correlation" : corr}, index=data.columns)
    
    
    
    
    
    
    

CPU times: total: 17.7 s
Wall time: 18.6 s


In [14]:
lin_score.sort_values(by='score_train', ascending=True).mean()

score_train    0.009246
score_test    -0.019002
benchmark     -0.021214
p_value        0.232417
correlation    0.029035
dtype: float64

In [18]:
lin_score.to_csv("LIN7score.csv")
lin_pred.to_csv("LIN7pred.csv")